In [1]:
# importing libnaries
import yfinance as yf
import numpy as np
import pandas as pd


In [2]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0].Symbol.tolist()

In [3]:
price_df = yf.download(tickers, start = '2015-01-01', auto_adjust=False)

[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (1d 2015-01-01 -> 2025-04-14)')
['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')


In [17]:
price_df[price_df.columns[price_df.columns.get_level_values(1) == 'NVDA']]

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,NVDA,NVDA,NVDA,NVDA,NVDA,NVDA
Date,,,,,,
2015-01-02,0.483099,0.503250,0.507000,0.495250,0.503250,113680000
2015-01-05,0.474939,0.494750,0.504750,0.492500,0.503250,197952000
2015-01-06,0.460540,0.479750,0.496000,0.479250,0.495500,197764000
2015-01-07,0.459340,0.478500,0.487500,0.477000,0.483250,321808000
2015-01-08,0.476619,0.496500,0.499500,0.483750,0.484000,283780000
...,...,...,...,...,...,...
2025-04-08,96.300003,96.300003,105.849998,94.459999,103.809998,476243400
2025-04-09,114.330002,114.330002,115.099998,97.529999,98.889999,612918300


In [18]:
# Creating the slicer function
def slice_df(symbol):
    sliced = price_df.copy()
    sliced = sliced[sliced.columns[sliced.columns.get_level_values(1)== symbol]]
    sliced.columns = sliced.columns.droplevel(1)
    sliced.loc[:, 'price'] = sliced.Open.shift(-1)
    return sliced

In [19]:
# Calling the function
test = slice_df('AAPL')

In [20]:
# Creating the movaing average function
def ma_calc(df, n, m): #n short term window, m long term window
    df['sma_1'] = df.Close.rolling(n).mean()
    df['sma_2'] = df.Close.rolling(m).mean()

In [32]:
def backtest(df, n, m):
    # Assume ma_calc adds 'sma_1' and 'sma_2' columns to df
    ma_calc(df, n, m)

    in_position = False
    profits = []

    for index, row in df.iterrows():
        # Entry signal
        if not in_position and row.sma_1 > row.sma_2:
            buy_price = row.price
            in_position = True

        # Exit signal
        elif in_position and row.sma_1 < row.sma_2:
            sell_price = row.price
            profit = (sell_price - buy_price) / buy_price
            profits.append(profit)
            in_position = False

    # If still in a position at the end, you might want to close it:
    if in_position:
        sell_price = df.iloc[-1].price
        profit = (sell_price - buy_price) / buy_price
        profits.append(profit)

    # Cumulative gain
    gain = (pd.Series(profits) + 1).prod()
    return gain


In [33]:
backtest(test, 30,60)

1.97638878744181

In [34]:
(test.sma_1>test.sma_2).idxmax()

Timestamp('2015-03-30 00:00:00')

In [35]:
test.index == (test.sma_1>test.sma_2).idxmax()

array([False, False, False, ..., False, False, False])

In [36]:
# Converting the index into pd series
first_buy = pd.Series(test.index == (test.sma_1 > test.sma_2).idxmax(), index=test.index)

In [ ]:
# true_signals = test[test == True].index[:5]
# true_signals

DatetimeIndex(['2015-01-02', '2015-01-05', '2015-01-06', '2015-01-07',
               '2015-01-08'],
              dtype='datetime64[ns]', name='Date', freq=None)

In [ ]:
(test.sma_1 > test.sma_2)